In [5]:
import requests
from datetime import datetime, timedelta
import xml.etree.ElementTree as ET
import pandas as pd
import csv
import sys


def fetch_data(from_date, to_date):
    headers = {
        'Content-Type': 'text/xml; charset=utf-8',
        'SOAPAction': 'http://web.cbr.ru/KeyRateXML',
    }

    body = f'''<?xml version="1.0" encoding="utf-8"?> 
    <soap:Envelope xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/"> 
        <soap:Body> 
            <KeyRateXML xmlns="http://web.cbr.ru/"> 
                <fromDate>{from_date}</fromDate> 
                <ToDate>{to_date}</ToDate> 
            </KeyRateXML> 
        </soap:Body> 
    </soap:Envelope> 
    '''

    response = requests.post('http://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx', headers=headers, data=body)

    if response.status_code == 200:
        return ET.fromstring(response.content)
    else:
        response.raise_for_status()


def parse_response(root):
    return [
        {
            'date': item.find('DT').text,
            'rate': float(item.find('Rate').text.replace(',', '.'))
        }
        for item in root.iter('KR')
    ]


def create_dataframe(data):
    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['date']).dt.date
    return df


def fill_missing_dates(df, start_date, end_date):
    all_dates = pd.date_range(start=start_date, end=end_date).date
    df = df.set_index('date').reindex(all_dates, method='ffill').reset_index()
    df.columns = ['date', 'rate']
    return df


def save_to_csv(df):
    df.to_csv(sys.stdout, index=False, encoding='UTF-8', quoting=csv.QUOTE_NONNUMERIC, sep=";")


def main():
    today = datetime.now()
    first_day_of_month = today.replace(day=1)
    from_date = first_day_of_month.strftime('%Y-%m-%d')
    to_date = today.strftime('%Y-%m-%d')

    root = fetch_data(from_date, to_date)
    data = parse_response(root)
    df = create_dataframe(data)

    df = fill_missing_dates(df, first_day_of_month.date(), today.date())

    df = df.sort_values(by='date')
    save_to_csv(df)


if __name__ == "__main__":
    main()

"date";"rate"
"2024-06-01";16.0
"2024-06-02";16.0
"2024-06-03";16.0
"2024-06-04";16.0
"2024-06-05";16.0
"2024-06-06";16.0
"2024-06-07";16.0
"2024-06-08";16.0
"2024-06-09";16.0
"2024-06-10";16.0
"2024-06-11";16.0
"2024-06-12";16.0
"2024-06-13";16.0
"2024-06-14";16.0
"2024-06-15";16.0
"2024-06-16";16.0
"2024-06-17";16.0
"2024-06-18";16.0
"2024-06-19";16.0
"2024-06-20";16.0
